# Описание
Определение автора текста из переписке в вайбере. Данные:


* Количество участников переписки: **3**
* Количество сообщений: **1815** в учебной выборке, **171** в тестовой
* **Эксперимент 1** - анализ каждого сообщения по-отдельности
* **Эксперимент 2** - анализ полной переписки пользователя в тестовой выборке.



In [ ]:
from google.colab import files # Для работы с файлами 
import numpy as np # Для работы с данными 
import pandas as pd # Для работы с таблицами
import matplotlib.pyplot as plt # Для вывода графиков
import os # Для работы с файлами
%matplotlib inline

from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.models import Sequential # Полносвязная модель
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation # Слои для сети
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами и преобразования их в последовательности
from tensorflow.keras.preprocessing.sequence import pad_sequences # Метод для работы с последовательностями
from tensorflow.keras.optimizers import Adam, Adadelta # оптимизаторы
from tensorflow.keras.callbacks import ModelCheckpoint, Callback

from sklearn.preprocessing import LabelEncoder # Метод кодирования тестовых лейблов
from sklearn.model_selection import train_test_split # Для разделения выборки на тестовую и обучающую

# Загружаем текстовый файл viber.text

In [ ]:
files.upload()

Saving viber.txt to viber (3).txt


{'viber.txt': b'[ 5 \xd0\xbc\xd0\xb0\xd1\x80\xd1\x82\xd0\xb0 2020 \xd0\xb3. 16:01 ] \xd0\xa7\xd1\x82\xd0\xbe \xd0\xb3\xd0\xbe\xd0\xb2\xd0\xbe\xd1\x80\xd1\x8f\xd1\x82?\r\n[ 5 \xd0\xbc\xd0\xb0\xd1\x80\xd1\x82\xd0\xb0 2020 \xd0\xb3. 16:04 ] \xd0\x9c\xd0\xb8\xd1\x85\xd0\xb0\xd0\xb8\xd0\xbb - \xd0\x9c\xd0\xb0\xd0\xba\xd0\xb0\xd1\x80\xd0\xb5\xd0\xbd\xd0\xba\xd0\xbe: \xd0\x91\xd0\xb5\xd1\x81\xd0\xbf\xd0\xbb\xd0\xb0\xd1\x82\xd0\xbd\xd0\xbe.\r\n[ 5 \xd0\xbc\xd0\xb0\xd1\x80\xd1\x82\xd0\xb0 2020 \xd0\xb3. 16:04 ] \xd0\x9c\xd0\xb8\xd1\x85\xd0\xb0\xd0\xb8\xd0\xbb - \xd0\x9c\xd0\xb0\xd0\xba\xd0\xb0\xd1\x80\xd0\xb5\xd0\xbd\xd0\xba\xd0\xbe: \xd0\x97\xd0\xb0\xd1\x80\xd0\xb5\xd0\xb3\xd0\xb8\xd1\x81\xd1\x82\xd1\x80\xd0\xb8\xd1\x80\xd1\x83\xd0\xb9\xd1\x81\xd1\x8f \xd0\xbf\xd0\xbe \xd1\x81\xd1\x81\xd1\x8b\xd0\xbb\xd0\xba\xd0\xb5 \xd0\xb8 \xd0\xb2\xd0\xbf\xd0\xb5\xd1\x80\xd1\x91\xd0\xb4\r\n[ 5 \xd0\xbc\xd0\xb0\xd1\x80\xd1\x82\xd0\xb0 2020 \xd0\xb3. 16:05 ] \xd0\x91\xd0\xbb\xd0\xb8\xd0\xbd, \xd0\xb3\xd0\xb4\

In [ ]:
with open('/content/viber.txt') as f:
  # Считываем файл и разбиваем построчно в список
  text_list = f.read().splitlines()

print(text_list[0])

[ 5 марта 2020 г. 16:01 ] Что говорят?


# Создаём массив из текстового файла
Внутри файла есть строка "#####" - которая разделяем тестовые и проверочные данные

In [ ]:
def create_array(text_list):
  # Сначала данные занесём в списки
  x_a_train_list = []
  x_m_train_list = []
  x_t_train_list = []
  x_a_test_list = []
  x_m_test_list = []
  x_t_test_list = []

  # Строки для идентификации и разбиения
  a_str = '] '
  m_str = '] Михаил - Макаренко:'
  t_str = '] Толик Бень:'

  current_list = ''  # Текуший список (для случая переноса текста на другую строку)
  mode = 'train'   # Режим добавления в список

  for i in range(len(text_list)):
    if '#####' in text_list[i]:
      mode = 'test'
      continue

    if m_str in text_list[i]:
      t = text_list[i].split(m_str)
      if (mode == 'train'):
        x_m_train_list.append(t[1])
        current_list = x_m_train_list
      if (mode == 'test'):
        x_m_test_list.append(t[1])
        current_list = x_m_test_list
    elif t_str in text_list[i]:
      t = text_list[i].split(t_str)
      if (mode == 'train'):
        x_t_train_list.append(t[1])
        current_list = x_t_train_list
      if (mode == 'test'):
        x_t_test_list.append(t[1])
        current_list = x_t_test_list
      current_list = t_list
    elif '] ' in text_list[i]:
      t = text_list[i].split(a_str)
      if (mode == 'train'):
        x_a_train_list.append(t[1])
        current_list = x_a_train_list
      if (mode == 'test'):
        x_a_test_list.append(t[1])
        current_list = x_a_test_list
    else:
      current_list.append(text_list[i])

  y_a_train_list = [0]*len(x_a_train_list)
  y_m_train_list = [1]*len(x_m_train_list)
  y_t_train_list = [2]*len(x_t_train_list)

  y_a_test_list = [0]*len(x_a_test_list)
  y_m_test_list = [1]*len(x_m_test_list)
  y_t_test_list = [2]*len(x_t_test_list)

  print('TRAIN')
  print ('Алексей: ', len(x_a_train_list), len(y_a_train_list), '\n', x_a_train_list[0:3], '/n')
  print ('Михаил: ', len(x_m_train_list), len(y_m_train_list), '\n', x_m_train_list[0:3], '/n')
  print ('Анатолий: ', len(x_t_train_list), len(y_t_train_list), '\n', x_t_train_list[0:3], '/n')
  print('Итого', (len(x_a_train_list) + len(x_m_train_list) + len(x_t_train_list)), '\n')

  print('TEST')
  print ('Алексей: ', len(x_a_test_list), len(y_a_test_list), '\n', x_a_test_list[0:3], '/n')
  print ('Михаил: ', len(x_m_test_list), len(y_m_test_list), '\n', x_m_test_list[0:3], '/n')
  print ('Анатолий: ', len(x_t_test_list), len(y_t_test_list), '\n', x_t_test_list[0:3], '/n')
  print('Итого', (len(x_a_test_list) + len(x_m_test_list) + len(x_t_test_list)), '\n')


  x_train_list = []
  x_train_list.extend(x_a_train_list)
  x_train_list.extend(x_m_train_list)
  x_train_list.extend(x_t_train_list)

  x_test_list = []
  x_test_list.extend(x_a_test_list)
  x_test_list.extend(x_m_test_list)
  x_test_list.extend(x_t_test_list)

  y_train_list = []
  y_train_list.extend(y_a_train_list)
  y_train_list.extend(y_m_train_list)
  y_train_list.extend(y_t_train_list)

  y_test_list = []
  y_test_list.extend(y_a_test_list)
  y_test_list.extend(y_m_test_list)
  y_test_list.extend(y_t_test_list)

  x_train = np.array(x_train_list)
  x_test = np.array(x_test_list)
  y_train = np.array(y_train_list)
  y_test = np.array(y_test_list)

  train_text_a = ' '.join(x_a_train_list)
  train_text_m = ' '.join(x_m_train_list)
  train_text_t = ' '.join(x_t_train_list)

  test_text_a = ' '.join(x_a_test_list)
  test_text_m = ' '.join(x_m_test_list)
  test_text_t = ' '.join(x_t_test_list)

  print('x_train:', x_train.shape, y_train.shape, '\nx_test:', x_test.shape, y_test.shape)

  return x_train, x_test, utils.to_categorical(y_train), utils.to_categorical(y_test), train_text_a, train_text_m, train_text_t, test_text_a, test_text_m, test_text_t

In [ ]:
x_train, x_test, y_train, y_test, train_text_a, train_text_m, train_text_t, test_text_a, test_text_m, test_text_t = create_array(text_list)

TRAIN
Алексей:  312 312 
 ['Что говорят?', 'Блин, где ссылка', '?...'] /n
Михаил:  653 653 
 [' Бесплатно.', ' Зарегистрируйся по ссылке и вперёд', ' https://events.epam.com/events/sam_python_meetup'] /n
Анатолий:  850 850 
 [' Ему это не грозит)))))', ' Кстати... Опа... Пусть сам хантит!', ' Отличная идея!'] /n
Итого 1815 

TEST
Алексей:  74 74 
 ['Потрясающий мужик! Блин, у меня аж ладони вспотели.', 'Не могу дозвониться - пишу везде - С ДНЁМ РОЖДЕНЬЯ!!!', 'В общем следующий год - принесёт небывалые перемены в обществе. Главное что бы мы оставались в "белом". '] /n
Михаил:  29 29 
 [' Толь, привет!', 'Верхонки заказал! Спасибо тебе. 13-го числа заберу. Фото выложу )))', ' Молодцы! Ещё раз поздравляю!'] /n
Анатолий:  68 68 
 [' Доброго дня! Воистину возможно всё, мне нравится этот мир! https://youtu.be/_Xv0KiefFPw', ' Ага! Заценил(wink)', ' https://habr.com/ru/company/intersystems/blog/516344/'] /n
Итого 171 

x_train: (1815,) (1815,) 
x_test: (171,) (171,)


# Токенизация

In [ ]:
num_words = 2000

tokenizer = Tokenizer(
    num_words=num_words,
    filters='!"#$%&*+,-–—./…;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
    lower=True, 
    split=' ', 
    char_level=False, 
    oov_token='unknown'
)

tokenizer.fit_on_texts(x_train)
items = list(tokenizer.word_index.items())

m_train = tokenizer.texts_to_matrix(x_train)
m_test = tokenizer.texts_to_matrix(x_test)

print('train:', m_train.shape,  y_train.shape, '\n', m_train[0], y_train[0])
print('test:', m_test.shape,  y_test.shape)

train: (1815, 2000) (1815, 3) 
 [0. 0. 0. ... 0. 0. 0.] [1. 0. 0.]
test: (171, 2000) (171, 3)


# Модель

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=m_train.shape[1], activation="sigmoid"))
model.add(Dropout(0.38))
model.add(BatchNormalization())
model.add(Dense(128, activation="sigmoid"))
model.add(BatchNormalization())
model.add(Dense(128, activation="sigmoid"))
model.add(Dropout(0.38))
model.add(BatchNormalization())
model.add(Dense(3, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(
    filepath='/content/best.h5', 
    monitor='val_accuracy',
    mode='max',
    verbose=1, 
    save_best_only=True
)

history = model.fit(
    m_train, 
    y_train, 
    epochs=100,
    batch_size=32,
    validation_data=(m_test, y_test),
    callbacks=[checkpointer]
)

model.load_weights('/content/best.h5')
results = model.evaluate(m_test, y_test, batch_size=32)

print('\nРЕЗУЛЬТАТ:')
print('test loss, test acc:', results)

Epoch 1/100
57/57 [==============================] - 2s 13ms/step - loss: 1.5541 - accuracy: 0.3482 - val_loss: 1.3485 - val_accuracy: 0.3977

Epoch 00001: val_accuracy improved from -inf to 0.39766, saving model to /content/best.h5
Epoch 2/100
57/57 [==============================] - 0s 6ms/step - loss: 1.3515 - accuracy: 0.3922 - val_loss: 1.3368 - val_accuracy: 0.3977

Epoch 00002: val_accuracy did not improve from 0.39766
Epoch 3/100
57/57 [==============================] - 0s 6ms/step - loss: 1.2231 - accuracy: 0.4185 - val_loss: 1.3051 - val_accuracy: 0.3977

Epoch 00003: val_accuracy did not improve from 0.39766
Epoch 4/100
57/57 [==============================] - 0s 6ms/step - loss: 1.2195 - accuracy: 0.4023 - val_loss: 1.2743 - val_accuracy: 0.3977

Epoch 00004: val_accuracy did not improve from 0.39766
Epoch 5/100
57/57 [==============================] - 0s 6ms/step - loss: 1.1221 - accuracy: 0.4428 - val_loss: 1.2602 - val_accuracy: 0.3977

Epoch 00005: val_accuracy did not 

# Вывод: 
Точность предсказания выше, чем при случайном угадывании, но недостаточная для точной идентификации.

# ======================================================================

# Эксперимент 2
Попробуем сделать предсказание, объедение все сообщения от одного пользователя в один текст. Наш эксперимент будет анализировать не отдельные сообщения, а сразу все тестовые сообщения пользователя, заключенные в одину строку.

In [ ]:
x2_test = np.array([test_text_a, test_text_m, test_text_t])
y2_test = utils.to_categorical(np.array([0, 1, 2]))

m2_test = tokenizer.texts_to_matrix(x2_test)

results = model.evaluate(m2_test, y2_test, batch_size=32)

print('\nРЕЗУЛЬТАТ:')
print('test loss, test acc:', results)

1/1 [==============================] - 0s 18ms/step - loss: 0.0778 - accuracy: 1.0000

РЕЗУЛЬТАТ:
test loss, test acc: [0.07784951478242874, 1.0]


# Выводы
По одному сообщению пользователя - небольшая вероятность классификации, по нескольким сообщениям (в данном примере за последние полтора месяца) - точность определения пользователя - 100%.

Подумал, что это возможно ошибка и поменял порядок следования всех текстов в массиве x2_test. И как полагается - результат 0. Это означает, что ошибки в нейроннойсети или в данных - нет.

Иными словами - для верификации пользователя по тексту нужно не одно сообщение, а несколько, (в моём случае - преписка за последние 1.5 месяца) - тогда персказание будет доходить до 100%.